# Modelo Baseline

Este notebook aplicará um algoritmo de *baseline* aos dados do arquivo **xtb_dataset.csv**.

**Autor:** Edélio Gabriel Magalhães de Jesus.

<a id='sumario'></a>
## Sumário

- [1) Introdução](#intro)
  - [1.1) O que é um modelo baseline?](#oqbaseline)
  - [1.2) Escolha do baseline](#escolha)
- [2) Aplicando o Baseline](#aplicando)
- [3) Conclusão](#conclu)
- [Referências](#ref)

<a id='intro'></a>
## 1) Introdução

<a id='oqbaseline'></a>
### **1.1) O que é um modelo baseline?**

Um **modelo baseline** é um modelo simples usado como ponto de partida num problema de aprendizado de máquina. Ele serve para dar uma referência de desempenho abaixo da qual qualquer modelo mais complexo precisa melhorar para que valha a pena [1].

**Por que usar um modelo baseline?**

- Ele fornece uma base de comparação para modelos mais sofisticados: se meu modelo “avançado” não superar o baseline, é sinal de que algo está errado. 
  
- Define um requisito mínimo de desempenho: qualquer modelo útil deve superar o baseline para justificar a complexidade. 
    
- Ajuda nas decisões de alocação de recursos, seleção de modelo e melhoria de dados: se o baseline já for bom o suficiente, talvez não precise investir tanto em modelos complexos. 

**Exemplos de estratégias de baseline para regressão**  

Algumas estratégias comuns para construir um baseline em tarefas de **regressão** são:  

- **“mean”**: prever sempre o valor médio das observações no conjunto de treino. É a estratégia mais comum e simples para regressão.  
  
- **“median”**: prever sempre o valor mediano das observações de treino, sendo mais robusta a outliers.  
  
- **“quantile”**: prever um quantil específico dos valores de treino (por exemplo, o percentil 90), útil quando o objetivo é avaliar o desempenho em faixas específicas da distribuição.  
  
- **“constant”**: prever sempre um valor fixo especificado previamente (por exemplo, 0 ou outro valor de referência).  


<a id='escolha'></a>
### **1.2 Escolha do baseline**

Escolheremos o modelo **DummyRegressor**.

---

O **DummyRegressor** é um modelo simples de regressão usado como referência (baseline) em problemas de aprendizado de máquina. Ele não aprende padrões nos dados, mas faz previsões usando estratégias básicas, como sempre prever a média, a mediana ou um valor constante dos dados de treino. Seu objetivo é servir como comparação: se um modelo mais complexo não consegue superar o desempenho do DummyRegressor, isso indica que ele não está realmente aprendendo relações significativas nas variáveis de entrada.

<a id='aplicando'></a>
## 2) Aplicando o Baseline

In [3]:
import pandas as pd
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.model_selection import train_test_split
import plotly.express as px

In [2]:
SEMENTE_ALEATORIA = 367
TAMANHO_TESTE=0.2
df_final = pd.read_csv('../dataset_processing/xtb_dataset.csv').sample(frac=0.1, random_state=SEMENTE_ALEATORIA)


X = df_final.drop(columns=['Delta'])
y = df_final['Delta']


# Divisão dos dados
X_treino_final, X_teste_final, y_treino_final, y_teste_final = train_test_split(
    X, y, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA
)

# Criação e treinamento do modelo baseline
dummy = DummyRegressor(strategy='mean')  # Usa a média das observações como previsão
dummy.fit(X_treino_final, y_treino_final)

# Avaliação no conjunto de teste
y_pred_dummy = dummy.predict(X_teste_final)
rmse_dummy = np.sqrt(mean_squared_error(y_teste_final, y_pred_dummy))

print(f"RMSE do modelo baseline (DummyRegressor): {rmse_dummy:.4f}")


RMSE do modelo baseline (DummyRegressor): 2.1159


Utilizamos a estretégia de média para prever os resultados. 

Perceba que já com o baseline obtivemos um valor baixo de RMSE. Mas precisamos levar em conta outros fatores como o intervalo dos valores do nosso *target*.

In [5]:
# Transformar para formato long
df_long = df_final.melt(var_name='Feature', value_name='Value')

# Criar boxplot 
fig = px.box(
    df_long, 
    x='Feature', 
    y='Value', 
    points='outliers',  # mostrar apenas outliers
    color='Feature',     # cores diferentes para cada feature
    title="Boxplots das Features e do Target",
    color_discrete_sequence=px.colors.qualitative.Dark2
)

# Ajustes estéticos
fig.update_layout(
    xaxis_title="Colunas",
    yaxis_title="Valores",
    boxmode='group',      # todos no mesmo eixo
    template='plotly_white',  # fundo branco limpo
    font=dict(family="Arial", size=12),
    showlegend=False
)

fig.update_traces(marker=dict(size=4))  # tamanho dos pontos outliers

fig.show()


Perceba nesse conjunto de *boxplots* que tanto as *features* como o *target* não possuem escalas muito diferentes, com exceção do atributo *ZPE*. 

Assim, um RMSE com valor baixo já era esperado. A própria natureza das amostras também colabora para isso, visto que estamos trabalhando com moléculas pequenas e, portanto, com atributos similares.

<a id='conclu'></a>
## 3) Conclusão

Aplicar e analisar o modelo *Baseline* é sempre uma boa prática, quando desejamos descobrir se nossos desempenhos, quando ótimos, realmente são resultado de um bom desenvolvimento de modelo ou é devido às próprias características do conjunto de dados com o qual nos propomos a trabalhar.

Assim, ao desenvolver nossos modelos, esperamos obter desempenhos ainda menores, resultado de que de fato houve um `"Aprendizado de Máquina"`.

## Referências

[1] **Site:** Medium - [Entendendo os Modelos de Base em Aprendizado de Máquina](https://medium.com/@preethi_prakash/understanding-baseline-models-in-machine-learning-3ed94f03d645)